In [2]:
import pandas as pd
import torch
import os
from transformers import T5Tokenizer, T5ForConditionalGeneration,Adafactor

In [2]:
train_df=pd.read_csv('webNLG2020_train.csv', index_col=[0])
train_df=train_df.iloc[  :35000,:]
train_df=train_df.sample(frac = 1)
batch_size=8
num_of_batches=int(len(train_df)/batch_size)

In [3]:
num_of_batches

4375

In [4]:
train_df

,prefix,input_text,target_text
21864,webNLG,Bananaman | starring | Tim_Brooke-Taylor && Ba...,The BBC began broadcasting Bananaman on 10/03/...
29613,webNLG,14th_New_Jersey_Volunteer_Infantry_Monument | ...,The 14th New Jersey Volunteer Infantry Monumen...
32543,webNLG,Aleksandra_Kovač | associatedBand/associatedMu...,Aleksandra Kovac has the musical genre of pop ...
30958,webNLG,AIDAstella | christeningDate | 2013-03-16 && A...,The christening date of AIDAstella is 2013-03-...
8431,webNLG,11264_Claudiomaccone | averageSpeed | 18.29 (k...,Nikolai Chernykh discovered 11264 Claudiomacco...
4593,webNLG,Serie_A | champions | Juventus_F.C.,Juventus F.C. are former champions of Serie A.
19044,webNLG,Albennie_Jones | genre | Jazz && Albennie_Jone...,Albennie Jones is a solo singer and performs j...
21172,webNLG,Akeem_Priestley | club | Sheikh_Russel_KC && A...,Akeem Priestley's club is Harbour View F.C and...
34426,webNLG,AFC_Ajax_(amateurs) | ground | Sportpark_De_To...,AFC Ajax (amateurs)'s ground is Sportpark De T...
3663,webNLG,Antares_(rocket) | finalFlight | 2014-07-13,The final flight for the rocket Antares was on...


In [5]:
if torch.cuda.is_available():
    dev = torch.device("cuda:0")
    print("Running on the GPU")
else:
    dev = torch.device("cpu")
    print("Running on the CPU")

Running on the CPU


In [10]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base',
                                             return_dict=True)
#moving the model to GPU
model.to(dev)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dr

In [7]:
optimizer = Adafactor(model.parameters(),lr=1e-3,
                      eps=(1e-30, 1e-3),
                      clip_threshold=1.0,
                      decay_rate=-0.8,
                      beta1=None,
                      weight_decay=0.0,
                      relative_step=False,
                      scale_parameter=False,
                      warmup_init=False)

In [8]:
from IPython.display import HTML, display
def progress(loss,value, max=100):
    return HTML(""" Batch loss :{loss}
      <progress    
value='{value}'max='{max}',style='width: 100%'>{value}
      </progress>
             """.format(loss=loss,value=value, max=max))

In [9]:
#Sets the module in training mode
model.train()
num_of_epochs = 3

loss_per_10_steps=[]
for epoch in range(1,num_of_epochs+1):
    print('Running epoch: {}'.format(epoch))
  
    running_loss=0

    out = display(progress(1, num_of_batches+1), display_id=True)
    for i in range(num_of_batches):
        inputbatch=[]
        labelbatch=[]
        new_df=train_df[i*batch_size:i*batch_size+batch_size]
        for indx,row in new_df.iterrows():
            input = 'WebNLG: '+row['input_text']+'</s>' 
            labels = row['target_text']+'</s>'   
            inputbatch.append(input)
            labelbatch.append(labels)
        inputbatch=tokenizer.batch_encode_plus(inputbatch,padding=True,max_length=400,return_tensors='pt')["input_ids"]
        labelbatch=tokenizer.batch_encode_plus(labelbatch,padding=True,max_length=400,return_tensors="pt") ["input_ids"]
        inputbatch=inputbatch.to(dev)
        labelbatch=labelbatch.to(dev)

        # clear out the gradients of all Variables 
        optimizer.zero_grad()

        # Forward propogation
        outputs = model(input_ids=inputbatch, labels=labelbatch)
        loss = outputs.loss
        loss_num=loss.item()
        logits = outputs.logits
        running_loss+=loss_num
        if i%10 ==0:      
            loss_per_10_steps.append(loss_num)
        out.update(progress(loss_num,i, num_of_batches+1))

        # calculating the gradients
        loss.backward()

        #updating the params
        optimizer.step()
    
    running_loss=running_loss/int(num_of_batches)
    print('Epoch: {} , Running loss: {}'.format(epoch,running_loss))

Running epoch: 1


/home/alex/.local/lib/python3.8/site-packages/transformers/models/t5/tokenization_t5.py:185: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(
/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)


KeyboardInterrupt: 

In [55]:
# torch.save(model.state_dict(),'trained_model/pytorch_model.bin')

In [118]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('trained_model', 
                                                return_dict=True)
import re

def generate(text):
    texts = text.split(".")
    result = ""
    for txt in texts:
        model.eval()
        input_ids = tokenizer.encode("WebNLG:{} </s>".format(txt), 
                                   return_tensors="pt")  
        outputs = model.generate(input_ids)
        result += tokenizer.decode(outputs[0])
    result = re.sub('<pad>|</s>',"",result)
    return result